# Synthetic Radio Host Generator

In [1]:
import os
import sys
import subprocess
import logging

# ------------------ CONFIG ------------------
REPO_URL = "https://github.com/Procrastinator02/synthetic_radio_host.git"
REPO_DIR = "/content/synthetic_radio_host"
BRANCH_NAME = "elevenlabs"   # 🔁 change this to your branch
# --------------------------------------------

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

def run_cmd(cmd, cwd=None):
    """Run shell command safely and fail fast"""
    logger.info(f"Running: {' '.join(cmd)}")
    result = subprocess.run(
        cmd,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    if result.returncode != 0:
        logger.error(result.stderr)
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
    return result.stdout.strip()

# Clone or update repository
if not os.path.exists(REPO_DIR):
    logger.info(f"Cloning branch '{BRANCH_NAME}' from {REPO_URL}")
    run_cmd([
        "git", "clone",
        "--branch", BRANCH_NAME,
        "--single-branch",
        REPO_URL,
        REPO_DIR
    ])
    logger.info("Repository cloned successfully")
else:
    logger.info("Repository exists, updating branch")
    run_cmd(["git", "fetch", "origin"], cwd=REPO_DIR)
    run_cmd(["git", "checkout", BRANCH_NAME], cwd=REPO_DIR)
    run_cmd(["git", "pull", "origin", BRANCH_NAME], cwd=REPO_DIR)
    logger.info("Repository updated successfully")

# Add repo to PYTHONPATH
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)
    logger.info(f"Added {REPO_DIR} to Python path")


2026-01-02 15:57:20 - INFO - Repository exists, updating branch
2026-01-02 15:57:20 - INFO - Running: git fetch origin
2026-01-02 15:57:21 - INFO - Running: git checkout elevenlabs
2026-01-02 15:57:21 - INFO - Running: git pull origin elevenlabs
2026-01-02 15:57:21 - INFO - Repository updated successfully
2026-01-02 15:57:21 - INFO - Added /content/synthetic_radio_host to Python path


In [2]:
# ------------------ CONFIG ------------------
REPO_SRC_DIR = "/content/synthetic_radio_host/src"
REQUIREMENTS_FILE = "/content/synthetic_radio_host/requirements.txt"
DEPS_MARKER = "/content/.deps_installed"
# --------------------------------------------

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

def run_cmd(cmd):
    """Run shell command safely and fail fast"""
    logger.info(f"Running: {' '.join(cmd)}")
    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    if result.returncode != 0:
        logger.error(result.stderr)
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
    return result.stdout.strip()

# ---------------- Dependency Installation ----------------
if not os.path.exists(DEPS_MARKER):
    logger.info("Checking requirements.txt")

    if not os.path.exists(REQUIREMENTS_FILE):
        raise FileNotFoundError(f"Missing {REQUIREMENTS_FILE}")

    logger.info("Installing dependencies from requirements.txt")
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "--upgrade", "pip"
    ])
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "-r", REQUIREMENTS_FILE
    ])

    open(DEPS_MARKER, "w").close()
    logger.info("Dependencies installed successfully")
else:
    logger.info("Dependencies already installed — skipping")

# ---------------- PYTHONPATH Setup ----------------
if REPO_SRC_DIR not in sys.path:
    sys.path.insert(0, REPO_SRC_DIR)
    logger.info(f"Added {REPO_SRC_DIR} to Python path")
else:
    logger.info("src/ already present in Python path")

2026-01-02 15:57:21 - INFO - Dependencies already installed — skipping
2026-01-02 15:57:21 - INFO - Added /content/synthetic_radio_host/src to Python path


# Setup and Configuration

In [3]:
from radio_host_functions import (
    CONFIG,
    __version__,
    fetch_wikipedia_article,
    generate_script_prompt,
    generate_script,
    generate_audio_segments,
    combine_and_export_audio
)

from openai import OpenAI
from google.colab import userdata, files

# ---------------- Logging ----------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

# ---------------- API Keys ----------------
try:
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    ELEVENLABS_API_KEY = userdata.get("ELEVENLABS_API_KEY")

    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in Colab secrets")
    if not ELEVENLABS_API_KEY:
        raise ValueError("ELEVENLABS_API_KEY not found in Colab secrets")

    logger.info("API keys loaded successfully")

except Exception as e:
    logger.error(f"Failed to load API keys: {e}")
    raise

# ---------------- OpenAI Client ----------------
openai_client = OpenAI(api_key=OPENAI_API_KEY)
logger.info("OpenAI client initialized")

# ---------------- ElevenLabs (SDK-safe init) ----------------
try:
    # ✅ OLD SDK (most common in Colab)
    import elevenlabs
    elevenlabs.set_api_key(ELEVENLABS_API_KEY)
    elevenlabs_client = elevenlabs
    logger.info("ElevenLabs initialized (legacy SDK)")

except Exception as e:
    logger.error(f"Failed to initialize ElevenLabs: {e}")
    raise

logger.info(f"Synthetic Radio Host version: {__version__}")


2026-01-02 15:57:26 - INFO - API keys loaded successfully
2026-01-02 15:57:26 - INFO - OpenAI client initialized
2026-01-02 15:57:26 - INFO - ElevenLabs initialized (legacy SDK)
2026-01-02 15:57:26 - INFO - Synthetic Radio Host version: 1.0.1


In [4]:
from openai import OpenAI

# ---------------- Logging ----------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

# ---------------- OpenAI Client ----------------
try:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    logger.info("OpenAI client initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize OpenAI client: {e}")
    raise

# ---------------- ElevenLabs Client (version-safe) ----------------
try:
    try:
        # ✅ New SDK (if installed)
        from elevenlabs.client import ElevenLabs
        eleven_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
        logger.info("ElevenLabs initialized using new SDK")

    except ModuleNotFoundError:
        # ✅ Legacy SDK (default in Colab)
        import elevenlabs
        elevenlabs.set_api_key(ELEVENLABS_API_KEY)
        eleven_client = elevenlabs
        logger.info("ElevenLabs initialized using legacy SDK")

except Exception as e:
    logger.error(f"Failed to initialize ElevenLabs client: {e}")
    raise

logger.info("All API clients initialized successfully")


2026-01-02 15:57:26 - INFO - OpenAI client initialized successfully
2026-01-02 15:57:26 - INFO - ElevenLabs initialized using legacy SDK
2026-01-02 15:57:26 - INFO - All API clients initialized successfully


In [5]:
# ---------------- Config ----------------
WIKIPEDIA_TOPIC = "MS Dhoni"
MAX_LOG_CHARS = 300  # prevent noisy logs
# ---------------------------------------

logger.info(f"Fetching Wikipedia article for topic: {WIKIPEDIA_TOPIC}")

try:
    if not WIKIPEDIA_TOPIC or not isinstance(WIKIPEDIA_TOPIC, str):
        raise ValueError("WIKIPEDIA_TOPIC must be a non-empty string")

    wiki_text = fetch_wikipedia_article(WIKIPEDIA_TOPIC)

    if not wiki_text or not wiki_text.strip():
        raise RuntimeError(f"No content returned for topic: {WIKIPEDIA_TOPIC}")

    logger.info(
        f"Fetched Wikipedia article successfully | "
        f"Topic='{WIKIPEDIA_TOPIC}' | "
        f"Characters={len(wiki_text)}"
    )

    # Optional: preview first few characters (safe for logs)
    logger.debug(
        f"Wikipedia preview:\n{wiki_text[:MAX_LOG_CHARS]}..."
    )

except Exception as e:
    logger.error(f"Failed to fetch Wikipedia article for '{WIKIPEDIA_TOPIC}': {e}")
    raise



2026-01-02 15:57:26 - INFO - Fetching Wikipedia article for topic: MS Dhoni
2026-01-02 15:57:26 - INFO - Fetching Wikipedia article: MS Dhoni
2026-01-02 15:57:26 - INFO - Wikipedia: language=en, user_agent: SyntheticRadioHost/1.0 (Wikipedia-API/0.6.0; https://github.com/martin-majlis/Wikipedia-API/), extract_format=1
2026-01-02 15:57:26 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=MS Dhoni&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2026-01-02 15:57:27 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=MS Dhoni&explaintext=1&exsectionformat=wiki
2026-01-02 15:57:27 - INFO - Wikipedia content loaded (2500 chars)
2026-01-02 15:57:27 - INFO - Fetched Wikipedia article successfully | Topic='MS Dhoni' | Characters=2500


In [6]:
# ---------------- Config ----------------
MIN_SCRIPT_CHARS = 600     # quality guardrail
RETRY_DELAY_SEC = 2
MAX_RETRIES = 2
# ---------------------------------------

logger.info("Starting script prompt generation")

try:
    if not wiki_text or not wiki_text.strip():
        raise ValueError("wiki_text is empty or invalid")

    # Generate prompt
    HINGLISH_PROMPT = generate_script_prompt(wiki_text)
    logger.info("Script prompt generated successfully")

    if not HINGLISH_PROMPT or not HINGLISH_PROMPT.strip():
        raise RuntimeError("Generated prompt is empty")

except Exception as e:
    logger.error(f"Failed during prompt generation: {e}")
    raise

# ---------------- Script Generation ----------------
script = None
for attempt in range(1, MAX_RETRIES + 1):
    try:
        logger.info(f"Generating script (attempt {attempt}/{MAX_RETRIES})")

        script = generate_script(HINGLISH_PROMPT, openai_client)

        if not script or not script.strip():
            raise RuntimeError("Generated script is empty")

        if len(script) < MIN_SCRIPT_CHARS:
            raise RuntimeError(
                f"Generated script too short ({len(script)} chars)"
            )

        logger.info(
            f"Script generated successfully | Characters={len(script)}"
        )
        break

    except Exception as e:
        logger.warning(f"Script generation failed: {e}")


2026-01-02 15:57:27 - INFO - Starting script prompt generation
2026-01-02 15:57:27 - INFO - Script prompt generated successfully
2026-01-02 15:57:27 - INFO - Generating script (attempt 1/2)
2026-01-02 15:57:27 - INFO - Generating script with OpenAI
2026-01-02 15:57:36 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-02 15:57:36 - INFO - Script generated (35 lines)
2026-01-02 15:57:36 - INFO - Script generated successfully | Characters=1430


In [7]:
import logging

# ---------------- Config ----------------
MIN_SEGMENTS = 1
# ---------------------------------------

logger.info("Starting audio segment generation")

try:
    if not script or not isinstance(script, str) or not script.strip():
        raise ValueError("Script is empty or invalid")

    # ✅ Correct call (single argument)
    audio_segments = generate_audio_segments(script)

    if not audio_segments or not isinstance(audio_segments, list):
        raise RuntimeError("generate_audio_segments returned no audio segments")

    if len(audio_segments) < MIN_SEGMENTS:
        raise RuntimeError(
            f"Insufficient audio segments generated ({len(audio_segments)})"
        )

    logger.info(
        f"Audio segments generated successfully | Segments={len(audio_segments)}"
    )

except Exception as e:
    logger.error(f"Audio generation failed: {e}")
    raise


2026-01-02 15:57:36 - INFO - Starting audio segment generation
2026-01-02 15:57:36 - INFO - Generating audio for 18 lines
2026-01-02 15:57:36 - INFO - TTS line 1/18 | voice=pNInz6obpgDQGcFmaJgB
2026-01-02 15:57:37 - ERROR - Audio generation failed: This request exceeds your quota of 10000. You have 9 credits remaining, while 22 credits are required for this request.


RateLimitError: This request exceeds your quota of 10000. You have 9 credits remaining, while 22 credits are required for this request.

In [ ]:
# ---------------- Config ----------------
OUTPUT_FILE = CONFIG.get("OUTPUT_FILENAME")
# ---------------------------------------

logger.info("Starting audio combination and export")

try:
    if not audio_segments or not isinstance(audio_segments, list):
        raise ValueError("audio_segments is empty or invalid")

    if not OUTPUT_FILE or not isinstance(OUTPUT_FILE, str):
        raise ValueError("CONFIG['OUTPUT_FILENAME'] is missing or invalid")

    # Combine segments and export final audio
    combine_and_export_audio(
        audio_segments,
        OUTPUT_FILE,
        files_download=files.download
    )

    # Verify file creation
    if not os.path.exists(OUTPUT_FILE):
        raise RuntimeError(f"Output file was not created: {OUTPUT_FILE}")

    file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)

    logger.info(
        f"Audio file generated successfully | "
        f"File='{OUTPUT_FILE}' | "
        f"Size={file_size_mb:.2f} MB"
    )

except Exception as e:
    logger.error(f"Audio export failed: {e}")
    raise

